In [1]:
import matplotlib.pyplot as plt
import numpy as np 
from scipy.interpolate import RegularGridInterpolator

def integrate_particles(positions, velocity_field, X, Y, dt, num_steps, L, phase_field, gamma):
    num_particles = len(positions)
    positions_over_time = np.zeros((num_steps+1, num_particles, 2))
    positions_over_time[0] = positions
    psiy, psix = -1*diff_wrt_y(velocity_field), diff_wrt_x(velocity_field)
    phix, phiy = diff_wrt_x(phase_field), diff_wrt_y(phase_field)
    interpsiy, interphix, interpsix, interphiy = interpolate(psiy), interpolate(phix), interpolate(psix), interpolate(phiy)
    t = [i*dt for i in range(num_steps+1)]
    a = [interpsiy, interphix, interpsix, interphiy]
    answer = new_vector_rk4(fun, t, polynomial_u, polynomial_v, positions_over_time, num_particles, velocities_over_time, a)
    return answer

def polynomial_u(coords, a):
    interpsiy, interphix = a[0], a[1]
    #print(coords)
    #print("interpolation below")
    #print(interpsiy(coords))
    #interpsiy, interphix = a[0], a[1]
    return (1-gamma)*interpsiy(coords)+gamma*interphix(coords)

def polynomial_v(coords, a):
    interpsix, interphiy = a[2], a[3]
    return (1-gamma)*interpsix(coords)+gamma*interphiy(coords)

def diff_wrt_y(original_matrix): 
    nt, nx, ny = original_matrix.shape
    umatrix = np.copy(original_matrix)
    for i in range(nt):
        umatrix[i] = np.gradient(original_matrix[i], axis=1) 
    return umatrix

def diff_wrt_x(original_matrix):
    vmatrix = np.copy(original_matrix)
    nt, nx, ny = original_matrix.shape
    for i in range(nt):
        vmatrix[i] = np.gradient(original_matrix[i], axis=0) 
    return vmatrix

def interpolate(umatrix):

    t = umatrix.shape[0]
    time = [i for i in range(t)]
    interpolator = RegularGridInterpolator((time, X, Y), umatrix, method = "cubic")
    return interpolator

def new_vector_rk4(f, t, polynomial_u, polynomial_v, positions_over_time, num_particles, a):
    n = len(t)
    y = positions_over_time

    for i in range(n - 1):
        h = t[i+1] - t[i]
        k1 = f(t[i], y[i], polynomial_u, polynomial_v, num_particles, a)
        k2 = f(t[i] + 0.5*h, (y[i] + 0.5*h*k1) % (2*L), polynomial_u, polynomial_v, num_particles, a)
        k3 = f(t[i] + 0.5*h, (y[i] + 0.5*h*k2) % (2*L), polynomial_u, polynomial_v, num_particles, a)
        k4 = f(t[i+1], (y[i] + h*k3) % (2*L), polynomial_u, polynomial_v, num_particles, a)
        y[i+1] = y[i] + (h/6)*(k1 + 2*k2 + 2*k3 + k4)
    y = np.mod(y, 2*L)
    velx = [polynomial_u((t[i], y[i][0], y[i][1]), a) for i in range(len(y))]
    vely = [polynomial_u((t[i], y[i][0], y[i][1]), a) for i in range(len(y))]
    result = np.concatenate((y, velx), axis=1)
    result = np.concatenate((result, vely), axis=1)
    return result

def fun(t, y, polynomial_u, polynomial_v, num_particles, a):
    xlist, ylist = [i % (2*L) for i in y[:, 0]], [i % (2*L) for i in y[:, 1]]
    timecoords = t*np.ones(num_particles)
    coords = [[x, y, z] for x, y, z in zip(timecoords, xlist, ylist)]
    dxdt, dydt = polynomial_u(coords, a), polynomial_v(coords, a)
    dxdt, dydt = np.array(dxdt).reshape(-1, 1), np.array(dydt).reshape(-1, 1)
    result = np.concatenate((dxdt, dydt), axis=1)
    return result